In [1]:
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")

Libraries imported.


In [2]:

# --- Verify Keys (Optional Check) ---
print("API Keys Set:")
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"OpenAI API Key set: {'Yes' if os.environ.get('OPENAI_API_KEY') and os.environ['OPENAI_API_KEY'] != 'YOUR_OPENAI_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"Anthropic API Key set: {'Yes' if os.environ.get('ANTHROPIC_API_KEY') and os.environ['ANTHROPIC_API_KEY'] != 'YOUR_ANTHROPIC_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

# Configure ADK to use API keys directly (not Vertex AI for this multi-model setup)
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"


# @markdown **Security Note:** It's best practice to manage API keys securely (e.g., using Colab Secrets or environment variables) rather than hardcoding them directly in the notebook. Replace the placeholder strings above.

API Keys Set:
Google API Key set: Yes
OpenAI API Key set: No (REPLACE PLACEHOLDER!)
Anthropic API Key set: No (REPLACE PLACEHOLDER!)


In [3]:
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash-lite"

# Note: Specific model names might change. Refer to LiteLLM or the model provider's documentation.
MODEL_GPT_4O = "openai/gpt-4o"
MODEL_CLAUDE_SONNET = "anthropic/claude-3-sonnet-20240229"
MODEL_OLLAMA_GEMMA3 = "ollama/gemma3:latest"
MODEL_OLLAMA_PHI4_MINI = "ollama/phi4-mini:latest"
MODEL_OLLAMA_GRANITE_3_3 = "ollama/granite3.3:latest"

print("\nEnvironment configured.")


Environment configured.


In [4]:
# @title Define the get_weather Tool
def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city (e.g., "New York", "London", "Tokyo").

    Returns:
        dict: A dictionary containing the weather information.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'report' key with weather details.
              If 'error', includes an 'error_message' key.
    """
    # Best Practice: Log tool execution for easier debugging
    print(f"--- Tool: get_weather called for city: {city} ---")
    city_normalized = city.lower().replace(" ", "") # Basic input normalization

    # Mock weather data for simplicity
    mock_weather_db = {
        "newyork": {"status": "success", "report": "The weather in New York is sunny with a temperature of 25°C."},
        "london": {"status": "success", "report": "It's cloudy in London with a temperature of 15°C."},
        "tokyo": {"status": "success", "report": "Tokyo is experiencing light rain and a temperature of 18°C."},
    }

    # Best Practice: Handle potential errors gracefully within the tool
    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"Sorry, I don't have weather information for '{city}'."}

# Example tool usage (optional self-test)
print(get_weather("New York"))
print(get_weather("Paris"))

--- Tool: get_weather called for city: New York ---
{'status': 'success', 'report': 'The weather in New York is sunny with a temperature of 25°C.'}
--- Tool: get_weather called for city: Paris ---
{'status': 'error', 'error_message': "Sorry, I don't have weather information for 'Paris'."}


In [5]:
# @title Define the get_weather Tool
def get_weather_english(city: str) -> str:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city (e.g., "New York", "London", "Tokyo").

    Returns:
        a simple message in English
    """
    # Best Practice: Log tool execution for easier debugging
    print(f"--- Tool: get_weather called for city: {city} ---")
    city_normalized = city.lower().replace(" ", "") # Basic input normalization

    # Mock weather data for simplicity
    mock_weather_db = {
        "newyork": "The weather in New York is sunny with a temperature of 25°C.",
        "london": "It's cloudy in London with a temperature of 15°C.",
        "tokyo": "Tokyo is experiencing light rain and a temperature of 18°C.",
    }

    # Best Practice: Handle potential errors gracefully within the tool
    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return f"Sorry, I don't have weather information for '{city}'."

# Example tool usage (optional self-test)
print(get_weather("New York"))
print(get_weather("Paris"))

--- Tool: get_weather called for city: New York ---
{'status': 'success', 'report': 'The weather in New York is sunny with a temperature of 25°C.'}
--- Tool: get_weather called for city: Paris ---
{'status': 'error', 'error_message': "Sorry, I don't have weather information for 'Paris'."}


In [6]:
# @title Define the Weather Agent
# Use one of the model constants defined earlier
AGENT_MODEL = MODEL_GEMINI_2_0_FLASH # Starting with a powerful Gemini model

weather_agent = Agent(
    name="weather_agent_v1",
    model=AGENT_MODEL, # Specifies the underlying LLM
    description="Provides weather information for specific cities.", # Crucial for delegation later
    instruction="You are a helpful weather assistant. Your primary goal is to provide current weather reports. "
                "When the user asks for the weather in a specific city, "
                "you MUST use the 'get_weather' tool to find the information. "
                "Analyze the tool's response: if the status is 'error', inform the user politely about the error message. "
                "If the status is 'success', present the weather 'report' clearly and concisely to the user. "
                "Only use the tool when a city is mentioned for a weather request.",
    tools=[get_weather], # Make the tool available to this agent
)

print(f"Agent '{weather_agent.name}' created using model '{AGENT_MODEL}'.")

Agent 'weather_agent_v1' created using model 'gemini-2.0-flash-lite'.


In [7]:
# @title Setup Session Service and Runner

# --- Session Management ---
# Key Concept: SessionService stores conversation history & state.
# InMemorySessionService is simple, non-persistent storage for this tutorial.
session_service = InMemorySessionService()

# Define constants for identifying the interaction context
APP_NAME = "weather_tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=weather_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='weather_tutorial_app', User='user_1', Session='session_001'
Runner created for agent 'weather_agent_v1'.


In [ ]:
# @title Define Agent Interaction Function
import asyncio
from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner: Runner, user_id: str, session_id: str):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

In [9]:
# @title Run the Initial Conversation

# We need an async function to await our interaction helper
async def run_conversation():
    await call_agent_async("What is the weather like in London?", runner, USER_ID, SESSION_ID)
    await call_agent_async("How about Paris?", runner, USER_ID, SESSION_ID) # Expecting the tool's error message
    await call_agent_async("Tell me the weather in New York", runner, USER_ID, SESSION_ID)

# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_conversation()


>>> User Query: What is the weather like in London?
  [Event] Author: weather_agent_v1, Type: Event, Final: False, Content: parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id='adk-67991088-b6f6-4c41-9d6e-48101e218c47', args={'city': 'London'}, name='get_weather'), function_response=None, inline_data=None, text=None)] role='model'
--- Tool: get_weather called for city: London ---
  [Event] Author: weather_agent_v1, Type: Event, Final: False, Content: parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=FunctionResponse(id='adk-67991088-b6f6-4c41-9d6e-48101e218c47', name='get_weather', response={'status': 'success', 'report': "It's cloudy in London with a temperature of 15°C."}), inline_data=None, text=None)] role='user'
  [Event] Author: weather_agent_v1, Type: Event, Final: True, Content: parts=[Pa

In [ ]:
from dotenv import load_dotenv
MODEL_LITELLM_GEMINI_2_0_FLASH = "gemini/gemini-2.0-flash-lite"
load_dotenv()
os.environ["GEMINI_API_KEY"] = os.environ.get('GOOGLE_API_KEY')
# @title Define and Test Ollama Agent

# Make sure 'get_weather' function from Step 1 is defined in your environment.
# Make sure 'call_agent_async' is defined from earlier.

# --- Agent using Ollama ---
weather_agent_ollama = None # Initialize to None
runner_ollama = None      # Initialize runner to None

model = MODEL_OLLAMA_PHI4_MINI

try:
    weather_agent_ollama = Agent(
        name="weather_agent",
        # Key change: Wrap the LiteLLM model identifier
        model=LiteLlm(model=model),
        description="Uses the get_weather_english tool to provides weather information for a specific city.", 
        instruction="You are a helpful weather assistant. Your primary goal is to provide weather report for a city. "
                "\nWhen the user asks for the weather in a city, use the 'get_weather_english' tool to find the weather information."
                "\nAnalyse the response:"
                "\n- If the response doesn't have weather details, apologise to the user in a friendly tone. Do not call any further tools!"
                "\n- If the response has the weather details, give final response to the user in friendly tone. Do not call any further tools!\n",
        tools=[get_weather_english], 
    )
    
    print(f"Agent '{weather_agent_ollama.name}' created using model '{model}'.")

    # InMemorySessionService is simple, non-persistent storage for this tutorial.
    session_service_ollama = InMemorySessionService() # Create a dedicated service

    # Define constants for identifying the interaction context
    APP_NAME_OLLAMA = "weather_tutorial_app_ollama" # Unique app name for this test
    USER_ID_OLLAMA = "user_1_ollama"
    SESSION_ID_OLLAMA = "session_001_ollama" # Using a fixed ID for simplicity

    # Create the specific session where the conversation will happen
    session_ollama = session_service_ollama.create_session(
        app_name=APP_NAME_OLLAMA,
        user_id=USER_ID_OLLAMA,
        session_id=SESSION_ID_OLLAMA
    )
    print(f"Session created: App='{APP_NAME_OLLAMA}', User='{USER_ID_OLLAMA}', Session='{SESSION_ID_OLLAMA}'")

    # Create a runner specific to this agent and its session service
    runner_ollama = Runner(
        agent=weather_agent_ollama,
        app_name=APP_NAME_OLLAMA,       # Use the specific app name
        session_service=session_service_ollama # Use the specific session service
        )
    print(f"Runner created for agent '{runner_ollama.agent.name}'.")

    # --- Test the Ollama Agent ---
    print("\n--- Testing Ollama Agent ---")
    # Ensure call_agent_async uses the correct runner, user_id, session_id
    await call_agent_async(query = "What's the weather in Mumbai?",
                           runner=runner_ollama,
                           user_id=USER_ID_OLLAMA,
                           session_id=SESSION_ID_OLLAMA)

except Exception as e:
    print(f"❌ Could not create or run Ollama agent '{model}'. Check API Key and model name. Error: {e}")